<a href="https://colab.research.google.com/github/dsercam/TC033/blob/main/TC4033_Activity1b_ID44.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import pandas as pd
import requests
from io import StringIO

In [6]:
url = 'https://drive.google.com/uc?export=download&id=1ttUEqCDuNFIenzrr4kMCHkbvBIy9fHmy'
ASLTrainSet = requests.get(url)

url = 'https://drive.google.com/uc?export=download&id=1mbEOJQKvhw41DpBzBB9PSfPr3IsngfA7'
ASLValidationSet = requests.get(url)

In [7]:
trainingAsText = StringIO(ASLTrainSet.text)
validationAsText = StringIO(ASLValidationSet.text)

trainingDf = pd.read_csv(trainingAsText)
validationDf = pd.read_csv(validationAsText)

Training Dataset size:
</br> 21,552,175

In [14]:
trCount = trainingDf.size
trCount

21552175

Training Dataset size:
</br> 5,630,020

In [15]:
valCount = validationDf.size
valCount

5630020

In [19]:
totalSamples = trCount + valCount
(trCount*100)/totalSamples

79.2878389695902

In [21]:
totalSamples = trCount + valCount
(valCount*100)/totalSamples

20.712161030409796